In [1]:
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from keras.layers import Dense, Input, concatenate, GRU, LSTM
from keras import backend as K
from keras.utils import to_categorical
from keras import Model
from nltk.translate.bleu_score import sentence_bleu

Using TensorFlow backend.


In [2]:
iris = load_iris()
X = iris['data']
y = iris['target']
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [3]:
df = pd.DataFrame(X)

In [4]:
bin_labels = pd.read_csv('../../data/raw/rpart_bin_labels.csv', delimiter=",")

In [5]:
bin_labels = bin_labels.rename(columns={"Unnamed: 0": "label", "label_list": "bins"})

In [6]:
path_df = pd.read_csv('../../data/raw/rpart_paths.csv', delimiter=",")

In [7]:
path_df = path_df.drop(["Unnamed: 0"], axis=1)

In [8]:
test_data = pd.concat([df, path_df], axis=1)
# test_data = df

In [9]:
new_path = []
for i, val in test_data.iterrows():
    new_path.append(val['new_col'].split(sep=","))

In [10]:
test_data.head()

,0,1,2,3,new_col
0,-0.900681,1.019004,-1.340227,-1.315444,3G0
1,-1.143017,-0.131979,-1.340227,-1.315444,3G0
2,-1.385353,0.328414,-1.397064,-1.315444,3G0
3,-1.506521,0.098217,-1.283389,-1.315444,3G0
4,-1.021849,1.249201,-1.340227,-1.315444,3G0


In [11]:
[x.insert(0, 'S') for x in new_path]
[x.append('E') for x in new_path]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [13]:
new_path

[['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],
 ['S', '3G0', 'E'],


In [12]:
test_data['new_path'] = new_path

In [ ]:
3G1', '4E0', '3R0', '4D0
3G1', '4E1', '3Q1
a = [1,0,0,0]
b = [1,1,1]
c = [0]

In [14]:
test_data = test_data.drop(["new_col"], axis=1)

In [15]:
test_data.head()

,0,1,2,3,new_path
0,-0.900681,1.019004,-1.340227,-1.315444,"[S, 3G0, E]"
1,-1.143017,-0.131979,-1.340227,-1.315444,"[S, 3G0, E]"
2,-1.385353,0.328414,-1.397064,-1.315444,"[S, 3G0, E]"
3,-1.506521,0.098217,-1.283389,-1.315444,"[S, 3G0, E]"
4,-1.021849,1.249201,-1.340227,-1.315444,"[S, 3G0, E]"


In [16]:
paths_lengths = np.array([len(xi)
                          for xi in test_data.iloc[:,-1]])

In [17]:
paths_lengths
np.max(paths_lengths)

7

In [18]:
label_char = []
for _, i in enumerate(np.unique(test_data['new_path'])):
    for _, j in enumerate(i):
        if j not in label_char:
            label_char.append(j)

In [19]:
label_indices = { j : i for i, j in enumerate(label_char) }

In [20]:
label_indices

{'S': 0,
 '3G0': 1,
 'E': 2,
 '3G1': 3,
 '4E0': 4,
 '3R0': 5,
 '4D0': 6,
 '4D1': 7,
 '3R1': 8,
 '4C0': 9,
 '4C1': 10,
 '1Y0': 11,
 '1Y1': 12,
 '4E1': 13,
 '3Q0': 14,
 '1V0': 15,
 '1V1': 16,
 '3Q1': 17}

In [21]:
indices_label = { i : j for i, j in enumerate(label_char) }
indices_label

{0: 'S',
 1: '3G0',
 2: 'E',
 3: '3G1',
 4: '4E0',
 5: '3R0',
 6: '4D0',
 7: '4D1',
 8: '3R1',
 9: '4C0',
 10: '4C1',
 11: '1Y0',
 12: '1Y1',
 13: '4E1',
 14: '3Q0',
 15: '1V0',
 16: '1V1',
 17: '3Q1'}

In [22]:
input_path_sequence = []
next_chars = []
features = []
paths_maxlen = np.max(paths_lengths)
path_vocab_size = len(bin_labels)
feature_size = 4
for i in range(0, len(test_data)):
    # get the feature
    curr_feat = np.array([test_data.iloc[i, 0:4]])
    curr_path = test_data.iloc[i, -1]
    curr_path_len = len(curr_path)
    # curr_label = y[i]
    # curr_dec_feat = df.iloc[i, 6]
    for j in range(1, curr_path_len):
        features.append(curr_feat)
        input_path_sequence.append(curr_path[0:j])
        next_chars.append(curr_path[j])

x_path = np.zeros(
    (len(input_path_sequence), paths_maxlen, path_vocab_size), dtype=np.bool)

path_latent_input = np.zeros(
    (len(input_path_sequence), feature_size), dtype=np.float)

y_path = np.zeros(
    (len(input_path_sequence), path_vocab_size), dtype=np.bool)

# print(input_path_sequence)
# print(len(input_path_sequence))
for i, sentence in enumerate(input_path_sequence):
    for t, char in enumerate(sentence):
        # x_path[i, t, self.char_indices[char]] = 1
        # print(bin_labels.index[bin_labels['label'] == char[1]])
        # index = bin_labels.index[bin_labels['label'] == char[1]].tolist()[0]
        x_path[i, t, label_indices[char]] = 1
    # y_path[i, char_indices[next_chars[i]]] = 1
    # index = bin_labels.index[bin_labels['label'] == next_chars[i][1]].tolist()[0]
    # y_path[i, index] = 1
    y_path[i, label_indices[next_chars[i]]] = 1
    path_latent_input[i, :] = features[i]
    
## Trouble with "S" and "E" index values.

In [23]:
def _create_label_model(latent_dim=5):
    input_layer = Input(shape=(feature_size,), name='ip_x')
    hidden_layer_x1 = Dense(20, activation='tanh',
                            name='hidden_x1')(input_layer)
    hidden_layer_x2 = Dense(20, activation='tanh',
                            name='hidden_x2')(hidden_layer_x1)
    hidden_layer_x3 = Dense(latent_dim, activation='tanh',
                            name='hidden_x3')(hidden_layer_x2)
    output_layer = Dense(len(np.unique(y)), activation='softmax',
                         name='op_x')(hidden_layer_x3)
    model = Model(input_layer, output_layer)
    return model

def _create_combined_model(initialize=True, rnn_cell='gru', latent_dim=5):

    label_model_latent = Input(shape=(latent_dim,), name='label_ip')
    path_input = Input(shape=(
        paths_maxlen, path_vocab_size), name='dec_feat_ip')
    if rnn_cell == 'gru':
        RNN = GRU
    else:
        RNN = LSTM

    decoder = RNN(latent_dim, return_state=False,
                  return_sequences=False, name='gru_seq')
    if initialize:
        decoder_outputs = decoder(
            path_input, initial_state=label_model_latent)
    else:
        decoder_outputs = decoder(path_input)

    merge_layer = concatenate(
        [label_model_latent, decoder_outputs], name='cat')
    output_chars = Dense(path_vocab_size,
                         activation='softmax', name='op_sent')(merge_layer)
    model = Model(
        [label_model_latent, path_input], output_chars)
    return model

In [24]:
combined_model = _create_combined_model()
label_model = _create_label_model()

Instructions for updating:
Colocations handled automatically by placer.


In [25]:
def get_hidden_x(x, model, layer_num=3):
    def get_hidden_x_inner(model, layer_num=layer_num):
        return K.function([model.layers[0].input], [model.layers[layer_num].output])
    return get_hidden_x_inner(model, layer_num=layer_num)([x])[0]

In [26]:
def fit_model():

    y_cat = to_categorical(y)

    label_model.compile(
        optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    label_model.fit(
        X, y_cat, batch_size=30, epochs=150, verbose=1, shuffle=True, validation_split=0.2)

    x_latent = get_hidden_x(path_latent_input, model=label_model)

    combined_model.compile(
        optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    combined_model.fit([x_latent, x_path], y_path,
                           batch_size=30, epochs=200, verbose=1, shuffle=True)

In [27]:
fit_model()

Instructions for updating:
Use tf.cast instead.
Train on 120 samples, validate on 30 samples
Epoch 1/150
120/120 [==============================] - 0s 2ms/step - loss: 0.9169 - acc: 0.6833 - val_loss: 1.1845 - val_acc: 0.0000e+00
Epoch 2/150
120/120 [==============================] - 0s 61us/step - loss: 0.8281 - acc: 0.6833 - val_loss: 1.1167 - val_acc: 0.0000e+00
Epoch 3/150
120/120 [==============================] - 0s 37us/step - loss: 0.7583 - acc: 0.7000 - val_loss: 1.0539 - val_acc: 0.0000e+00
Epoch 4/150
120/120 [==============================] - 0s 57us/step - loss: 0.6962 - acc: 0.7083 - val_loss: 0.9978 - val_acc: 0.0000e+00
Epoch 5/150
120/120 [==============================] - 0s 49us/step - loss: 0.6499 - acc: 0.7333 - val_loss: 0.9429 - val_acc: 0.0000e+00
Epoch 6/150
120/120 [==============================] - 0s 35us/step - loss: 0.6086 - acc: 0.7500 - val_loss: 0.8942 - val_acc: 0.1000
Epoch 7/150
120/120 [==============================] - 0s 31us/step - loss: 0.5765 -

120/120 [==============================] - 0s 53us/step - loss: 0.1153 - acc: 0.9917 - val_loss: 0.4842 - val_acc: 0.8333
Epoch 120/150
120/120 [==============================] - 0s 49us/step - loss: 0.1143 - acc: 0.9917 - val_loss: 0.4850 - val_acc: 0.8333
Epoch 121/150
120/120 [==============================] - 0s 45us/step - loss: 0.1135 - acc: 0.9917 - val_loss: 0.4775 - val_acc: 0.8667
Epoch 122/150
120/120 [==============================] - 0s 45us/step - loss: 0.1125 - acc: 0.9917 - val_loss: 0.4758 - val_acc: 0.8667
Epoch 123/150
120/120 [==============================] - 0s 46us/step - loss: 0.1121 - acc: 0.9917 - val_loss: 0.4872 - val_acc: 0.8000
Epoch 124/150
120/120 [==============================] - 0s 53us/step - loss: 0.1112 - acc: 0.9917 - val_loss: 0.4740 - val_acc: 0.8333
Epoch 125/150
120/120 [==============================] - 0s 50us/step - loss: 0.1100 - acc: 0.9917 - val_loss: 0.4681 - val_acc: 0.8667
Epoch 126/150
120/120 [==============================] - 0s 50

560/560 [==============================] - 0s 204us/step - loss: 1.6869 - acc: 0.3607
Epoch 41/200
560/560 [==============================] - 0s 203us/step - loss: 1.6827 - acc: 0.3589
Epoch 42/200
560/560 [==============================] - 0s 163us/step - loss: 1.6783 - acc: 0.3589
Epoch 43/200
560/560 [==============================] - 0s 189us/step - loss: 1.6744 - acc: 0.3589
Epoch 44/200
560/560 [==============================] - 0s 166us/step - loss: 1.6701 - acc: 0.3589
Epoch 45/200
560/560 [==============================] - 0s 166us/step - loss: 1.6657 - acc: 0.3607
Epoch 46/200
560/560 [==============================] - 0s 175us/step - loss: 1.6608 - acc: 0.3625
Epoch 47/200
560/560 [==============================] - 0s 154us/step - loss: 1.6558 - acc: 0.3625
Epoch 48/200
560/560 [==============================] - 0s 147us/step - loss: 1.6506 - acc: 0.3625
Epoch 49/200
560/560 [==============================] - 0s 147us/step - loss: 1.6444 - acc: 0.3661
Epoch 50/200
560/560 [=

In [28]:
combined_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
label_ip (InputLayer)           (None, 5)            0                                            
__________________________________________________________________________________________________
dec_feat_ip (InputLayer)        (None, 7, 26)        0                                            
__________________________________________________________________________________________________
gru_seq (GRU)                   (None, 5)            480         dec_feat_ip[0][0]                
                                                                 label_ip[0][0]                   
__________________________________________________________________________________________________
cat (Concatenate)               (None, 10)           0           label_ip[0][0]                   
          

In [29]:
def predict(x):
    latent_dim = 5
    x_f = x.reshape(1, feature_size)
    token = 'S'
    cont = True
    path = [token]
    x_path = np.zeros((1, paths_maxlen, path_vocab_size), dtype=np.bool)

    x_latent = get_hidden_x(x_f, model=label_model)
    x_latent = x_latent.reshape(1, latent_dim)
    x_path[0, 0, label_indices[token]] = 1
    pred = label_model.predict(x_f)
    label = [np.argmax(pred[0])]
    index = 1
    while cont & (index < paths_maxlen):
        pred = combined_model.predict([x_latent, x_path])
        char_index = np.argmax(pred[0])
        x_path[0, index, char_index] = 1
        next_char = indices_label[char_index]
        path.append(next_char)
        index += 1
        if next_char == 'E':
            cont = False
        # elif index == self.paths_maxlen - 1:
        #     path.append('E')

    if path[-1] != 'E':
        path.append('E')

    return [path, label]

In [30]:
def check_path(path): # Returns -1 if path traversed is wrong/non-existant
    # path = ''.join(path)
    path = path[1:-1]
    pred_features = []
    path_as_strings = []
    for i in range(len(path)):
        pred_features.append(int(path[i][:-1]))
        path_as_strings.append(path[i][-1])
        # if i%2 == 0:
        #     print('i -- ', i)
        #     print('path -- ', path)
        #     print('path[i] -- ', path[i])
        #     pred_features.append(int(path[i]))
        # else:
        #     path_as_strings.append(path[i])

    n_nodes = self.clf.tree_.node_count
    children_left = self.clf.tree_.children_left
    children_right = self.clf.tree_.children_right
    feature = self.clf.tree_.feature

    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, -1)]
    while len(stack) > 0:
        node_id, parent_depth = stack.pop()
        # node_depth[node_id] = parent_depth + 1

        if (children_left[node_id] != children_right[node_id]):
            stack.append((children_left[node_id], parent_depth + 1))
            stack.append((children_right[node_id], parent_depth + 1))
        else:
            is_leaves[node_id] = True


    node = 0
    pred_target = -1
    subset_path = False
    for i in range(len(path_as_strings)):
        if path_as_strings[i] == 'L':
            if feature[node]+1 == pred_features[i]:
                node = children_left[node]
            # else:
                # pred_target = -1 # Remove for "subset" checks
                # break
        elif path_as_strings[i] == 'R':
            if feature[node]+1 == pred_features[i]:
                node = children_right[node]
            # else:
                # pred_target = -1 # Remove for "subset" checks
                # break
        if is_leaves[node]:
            for i, x in enumerate(self.clf.tree_.value[node][0]):
                if x > 0:
                    pred_target = i
            if i < len(path_as_strings):
                subset_path = True

    return pred_target, subset_path

In [33]:
path_df

,new_col
0,3G0
1,3G0
2,3G0
3,3G0
4,3G0
5,3G0
6,3G0
7,3G0
8,3G0
9,3G0


In [31]:
def score():
    count = []
    bleu_score = []
    j_coeff = []
    l_dist = []
    path_mismatch_count = []
    traverse_check_count = []
    order_mismatch_count = []
    subset_path_count = []
    for i in range(test_data.shape[0]):
        curr_feat = np.array([test_data.iloc[i, 0:X.shape[1]]])
        path, label = predict(curr_feat)
        actual_path = test_data.iloc[i, -1]

        actual_path_tok = [label_indices[char] for char in actual_path]
        pred_path_tok = [label_indices[char] for char in path]

        # j_coeff.append(super().get_j_coeff(actual_path_tok, pred_path_tok))

        print('actual vs predicted: ', test_data.iloc[i, -1], ' vs ', ' '.join(
            path), 'labels: ', y[i], label[0])
        count.append(y[i] == label[0])
        # print('Actual path -- ', actual_path)
        # print('Pred path -- ', path)
#         if actual_path != path:
#             print(' -- Path mismatch -- ')
#             if sorted(actual_path) == sorted(path):
#                 print(' -- Order mismatch -- ')
#                 order_mismatch_count.append(1)
#             else:
#                 path_mismatch_count.append(1)
#                 pred_target, subset_path = self.check_path(path)
#                 subset_path_count.append(subset_path)
#                 if pred_target != -1 and pred_target == self.df.iloc[i, self.X.shape[1]+1]:
#                     traverse_check_count.append(1)


        path = list(''.join(path))
        actual_path = list(''.join(test_data.iloc[i, -1]))
        bleu_score.append(sentence_bleu([actual_path], path))

#         lev_path = []
#         for i in range(len(path)):
#             if i in ['S','L','R','E']:
#                 lev_path.append(i)
#         l_dist.append(distance.levenshtein(
#             self.df.iloc[i, self.X.shape[1]].replace(' ', ''), ''.join(lev_path)))


    print('\nLabel accuracy - ', np.mean(count))
#     print('Path metric (Jaccard) - ', np.mean(j_coeff))
#     print('Path metric (Levenshtein) - ', np.mean(l_dist))
#     print('Path mismatch count - ', np.sum(path_mismatch_count))
#     print('Right traverse count - ', np.sum(traverse_check_count))
#     print('Order mismatch count - ', np.sum(order_mismatch_count))
#     print('Subset path count - ', np.sum(subset_path_count))
    print('Bleu score of paths - ', np.mean(bleu_score))

In [32]:
score()

actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs predicted:  ['S', '3G0', 'E']  vs  S 3G0 E labels:  0 0
actual vs 

/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it cont

actual vs predicted:  ['S', '3G1', '4E1', '3Q1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3G1', '4E1', '3Q1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3G1', '4E1', '3Q1', 'E']  vs  S 3G1 4E1 3Q1 E labels:  2 2
actual vs predicted:  ['S', '3G1', '4E1', '3Q1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3G1', '4E1', '3Q1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3G1', '4E0', '3R0', '4D1', 'E']  vs  S 3G1 4E1 3Q1 E labels:  2 2
actual vs predicted:  ['S', '3G1', '4E1', '3Q1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3G1', '4E1', '3Q1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3G1', '4E1', '3Q1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3G1', '4E1', '3Q1', 'E']  vs  S 3G1 4E1 3Q1 E labels:  2 2
actual vs predicted:  ['S', '3G1', '4E1', '3Q1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3G1', '4E1', '3Q1', 'E']  vs  S E labels:  2 2
actual vs predicted:  ['S', '3G1', '4E1', '3Q

### Path invariance trials

In [4]:
## Import nnum, vnum, nodes, csplit, split_df,
## frame

splits = pd.read_csv('../../data/raw/splits.csv', delimiter=",", index_col=0)
csplit = pd.read_csv('../../data/raw/csplit.csv', delimiter=",")
frame = pd.read_csv('../../data/raw/frame.csv', delimiter=",")

frame = frame.drop(["Unnamed: 0"], axis=1)
frame = frame.rename(columns={"var": "variable"})
# bin_labels = bin_labels.rename(columns={"Unnamed: 0": "label", "label_list": "bins"})

In [5]:
## Generate nnum, vnum, nodes(split_df and csplit - 2L if necessary)

temp_frame = frame

nc = temp_frame[["ncompete", "nsurrogate"]]

index = np.cumsum((frame[["variable"]]!="<leaf>").values + nc[["ncompete"]].values + nc[["nsurrogate"]].values)

index_df = pd.DataFrame((np.insert(index,0,0)+1)[:-1], columns=["i"])

temp_frame = pd.concat([temp_frame, index_df], axis=1)

# temp_frame[temp_frame[["var"]]=="<leaf>"]
# temp_frame.where(temp_frame[["var"]]=="<leaf>")
# temp_frame.loc[temp_frame[["variable"]]=="<leaf>", "index"] = 0
temp_frame.i[temp_frame.variable == "<leaf>"] = 0

/home/shakkeel/anaconda3/envs/test_imly/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [6]:
temp_frame

,variable,n,wt,dev,yval,complexity,ncompete,nsurrogate,yval2.,yval2..1,yval2..2,yval2..3,yval2..4,yval2..5,yval2..6,yval2.nodeprob,i
0,Petal.Length,150,150,100,1,0.500,3,3,1.0,50.0,50.0,50.0,0.333333,0.333333,0.333333,1.000000,1
1,<leaf>,50,50,0,1,0.000,0,0,1.0,50.0,0.0,0.0,1.000000,0.000000,0.000000,0.333333,0
2,Petal.Width,100,100,50,2,0.440,3,3,2.0,0.0,50.0,50.0,0.000000,0.500000,0.500000,0.666667,8
3,Petal.Length,54,54,5,2,0.020,3,0,2.0,0.0,49.0,5.0,0.000000,0.907407,0.092593,0.360000,15
4,Petal.Width,48,48,1,2,0.010,3,0,2.0,0.0,47.0,1.0,0.000000,0.979167,0.020833,0.320000,19
5,<leaf>,47,47,0,2,0.000,0,0,2.0,0.0,47.0,0.0,0.000000,1.000000,0.000000,0.313333,0
6,<leaf>,1,1,0,3,0.000,0,0,3.0,0.0,0.0,1.0,0.000000,0.000000,1.000000,0.006667,0
7,Petal.Width,6,6,2,3,0.010,3,2,3.0,0.0,2.0,4.0,0.000000,0.333333,0.666667,0.040000,23
8,Sepal.Length,3,3,1,2,0.010,3,0,2.0,0.0,2.0,1.0,0.000000,0.666667,0.333333,0.020000,29
9,<leaf>,2,2,0,2,0.000,0,0,2.0,0.0,2.0,0.0,0.000000,1.000000,0.000000,0.013333,0


In [7]:
nodes = temp_frame[["n", "ncompete", "nsurrogate", "i"]]

In [8]:
nnum = list(temp_frame.index) # row names of temp_frame

In [9]:
feature_names = ["Sepal.Length", "Sepal.Width", "Petal.Length", "Petal.Width"]

vnum = list(map(feature_names.index, splits.index))

In [10]:
nodes

,n,ncompete,nsurrogate,i
0,150,3,3,1
1,50,0,0,0
2,100,3,3,8
3,54,3,0,15
4,48,3,0,19
5,47,0,0,0
6,1,0,0,0
7,6,3,2,23
8,3,3,0,29
9,2,0,0,0


In [11]:
nnum

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [23]:
def return_yval(path): # [1,0,0,0]
    node = 0
    nspl = 1
    i = 0
    while nspl != 0:
        npos = nnum[node] # i)0, 
        nspl = nodes.iloc[npos][3] # i)1
        var = vnum[nspl]
        # ncat
        temp = splits.iloc[nspl][3]
        if nspl > 0:
            print("nspl succeeded")
            if path[i] == 0: # i)1
                direction = -1
                i+=1
            else:
                direction = 1
                i+=1

            if direction == -1:
                if node == 0:
                    node = 1
                node = 2 * node
                print(node)
            else:
                if node == 0:
                    node = 1
                node = 2 * node + 1
                print(node)
            if len(path) == 1:
                nspl = 0
        else:
            print('nspl failed')
            yval = temp_frame.iloc[node][4]
    return temp_frame.iloc[node][4]

In [36]:

path = [1,1,0,1]
a = [1,0,0,0]
b = [1,1,1]
c = [0]
d = [1,1,1,1,1,1,0]
e = [1,0,1,0,0]
return_yval(e)

nspl succeeded
3
nspl succeeded
6
nspl failed


3

In [30]:
temp_frame

,variable,n,wt,dev,yval,complexity,ncompete,nsurrogate,yval2.,yval2..1,yval2..2,yval2..3,yval2..4,yval2..5,yval2..6,yval2.nodeprob,i
0,Petal.Length,150,150,100,1,0.500,3,3,1.0,50.0,50.0,50.0,0.333333,0.333333,0.333333,1.000000,1
1,<leaf>,50,50,0,1,0.000,0,0,1.0,50.0,0.0,0.0,1.000000,0.000000,0.000000,0.333333,0
2,Petal.Width,100,100,50,2,0.440,3,3,2.0,0.0,50.0,50.0,0.000000,0.500000,0.500000,0.666667,8
3,Petal.Length,54,54,5,2,0.020,3,0,2.0,0.0,49.0,5.0,0.000000,0.907407,0.092593,0.360000,15
4,Petal.Width,48,48,1,2,0.010,3,0,2.0,0.0,47.0,1.0,0.000000,0.979167,0.020833,0.320000,19
5,<leaf>,47,47,0,2,0.000,0,0,2.0,0.0,47.0,0.0,0.000000,1.000000,0.000000,0.313333,0
6,<leaf>,1,1,0,3,0.000,0,0,3.0,0.0,0.0,1.0,0.000000,0.000000,1.000000,0.006667,0
7,Petal.Width,6,6,2,3,0.010,3,2,3.0,0.0,2.0,4.0,0.000000,0.333333,0.666667,0.040000,23
8,Sepal.Length,3,3,1,2,0.010,3,0,2.0,0.0,2.0,1.0,0.000000,0.666667,0.333333,0.020000,29
9,<leaf>,2,2,0,2,0.000,0,0,2.0,0.0,2.0,0.0,0.000000,1.000000,0.000000,0.013333,0


In [33]:
splits

,count,ncat,improve,index,adj
Petal.Length,150,-1,50.000000,2.45,0.000000
Petal.Width,150,-1,50.000000,0.80,0.000000
Sepal.Length,150,-1,34.164050,5.45,0.000000
Sepal.Width,150,1,19.038508,3.35,0.000000
Petal.Width,0,-1,1.000000,0.80,1.000000
Sepal.Length,0,-1,0.920000,5.45,0.760000
Sepal.Width,0,1,0.833333,3.35,0.500000
Petal.Width,100,-1,38.969404,1.75,0.000000
Petal.Length,100,-1,37.353535,4.75,0.000000
Sepal.Length,100,-1,10.686869,6.15,0.000000


In [96]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')